In [1]:
# Dem Libraries!
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import Input, Reshape
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard
import pretty_midi
np.random.seed(77)

#Paths
training_path = './1000_Interpolated/'
total_samps = 19000

Using TensorFlow backend.


In [2]:
# To get the sine waves data
# n_samples : Number of sine waves you want generated
def sample_data(training_path):
    filenames = []
    lens = []
    fin_midi_stream = []
    for filename in os.listdir(training_path):
        if filename.endswith('.mid'):
            filenames.append(os.path.join(training_path,filename))
    fin_midi_streama = []
    fin_midi_files = []
    for file in filenames:
        temp = pretty_midi.PrettyMIDI(file)
        melody = temp.instruments[0]
    #     chord = temp.instruments[1]
        melody_math = melody.get_piano_roll(16)
        curr_stream = np.argmax(melody_math, axis=0)
        fin_midi_streama.extend(curr_stream)
    chop_size = 256
    num_of_chops = len(fin_midi_streama)//chop_size
    for i in range(num_of_chops):
        fin_midi_files.append(fin_midi_streama[(i)*chop_size:(i+1)*chop_size])
    return fin_midi_files

def UN_normalize_data(INP):
    OUTP = np.zeros((len(INP),len(INP[0])))
#     OUTP = INP
    for i in range(len(INP)):
        for j in range(len(INP[i])):
            OUTP[i][j]=(data_minim + (INP[i][j]*(data_maxim-data_minim)) )
    return OUTP

def normalize_data(INP):
    OUTP = np.zeros((len(INP),len(INP[0])))
#     OUTP = INP
    for i in range(len(INP)):
        for j in range(len(INP[i])):
            OUTP[i][j]=int((INP[i][j]-data_minim)/(data_maxim-data_minim))
    return OUTP

ALL_DATA = sample_data(training_path)
data_minim = min(min(ALL_DATA))
data_maxim = max(max(ALL_DATA))
ALL_NEW_DATA = normalize_data(ALL_DATA)

def get_ALL_data(n_samples):
    return ALL_NEW_DATA[:n_samples]
#     return ALL_DATA[:n_samples]

# Function to get the max and min notes
def get_max_min_notes(all_the_data):
    minim = 99999
    maxim = 0
    for i in range(len(all_the_data)):
        for j in all_the_data[i]:
            if j>maxim:
                maxim= j
            if j<minim:
                minim = j
    return maxim,minim

# Get Randomn Input( Uniform but now integers)
def get_random_in_here(n_samples,noise_dim):
    XN_noise = np.random.uniform(0, 1, size=[n_samples, noise_dim])
    for i in range(len(XN_noise)):
        for j in range(len(XN_noise[i])):
            XN_noise[i][j] = round(XN_noise[i][j])
    return XN_noise

In [3]:
#Generator Model :
# Needs to be able to take in random noise and generate realistic looking output data

def get_generative(G_in, dense_dim=1000, out_dim=256, lr=1e-3):
    x = Dense(dense_dim)(G_in)
    x = Activation('tanh')(x)
    G_out = Dense(out_dim, activation='tanh')(x)
    G = Model(G_in, G_out)
    opt = SGD(lr=lr)
    G.compile(loss='binary_crossentropy', optimizer=opt)
    return G, G_out

G_in = Input(shape=[16])
G, G_out = get_generative(G_in)
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              17000     
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               256256    
Total params: 273,256
Trainable params: 273,256
Non-trainable params: 0
_________________________________________________________________


In [4]:
#Discriminator Model :
# Needs to be able to take in a generated signal and distinguish between real and fake(generated)

def get_discriminative(D_in, lr=1e-3, drate=.25, n_channels=100, conv_sz=5, leak=.2):
    x = Reshape((-1, 1))(D_in)
    
    x = Conv1D(n_channels, conv_sz, activation='relu')(x)
    x = Dropout(drate)(x)
    x = Flatten()(x)
    x = Dense(n_channels)(x)
    D_out = Dense(2, activation='sigmoid')(x)
    D = Model(D_in, D_out)
    dopt = Adam(lr=lr)
    D.compile(loss='binary_crossentropy', optimizer=dopt)
    return D, D_out

D_in = Input(shape=[256])
D, D_out = get_discriminative(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 256, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 252, 100)          600       
_________________________________________________________________
dropout_1 (Dropout)          (None, 252, 100)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               2520100   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total para

In [5]:
# Function to make sure the Discriminators weights are frozen, while training the Generator
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

        
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in)
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

GAN_in = Input([16])
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 16)                0         
_________________________________________________________________
model_1 (Model)              (None, 256)               273256    
_________________________________________________________________
model_2 (Model)              (None, 2)                 2520902   
Total params: 2,794,158
Trainable params: 273,256
Non-trainable params: 2,520,902
_________________________________________________________________


In [6]:
# Function to get a concatenated input of real_data and fake_data with appropriate concatenated labels
def sample_data_and_gen(G, noise_dim=16, n_samples=total_samps):
    # Generated Real Samples 
    XT = get_ALL_data(n_samples)
    # Get the random noise that'll be used to generate the fake data
    #XN_noise = np.random.uniform(0, 108, size=[n_samples, noise_dim])
    XN_noise = get_random_in_here(n_samples,noise_dim)
    # Generate the Fake Samples
    XN = G.predict(XN_noise)
    # Concatenat the Real and Fake Data
    X = np.concatenate((XT, XN))
    # Concatenate the labels and set the appropriate values
    y = np.zeros((2*n_samples, 2))
    y[:n_samples, 1] = 1
    y[n_samples:, 0] = 1
    
    return X, y

def pretrain(G, D, noise_dim=16, n_samples=total_samps, batch_size=100):
    # Get the real samples for the pretraining of the discriminator
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    # Allow the Discriminator to be trained
    set_trainability(D, True)
    # Fit the Discriminator with 1 epoch
    D.fit(X, y, epochs=1, batch_size=batch_size)


In [7]:
# Perform One Train run for the Determiner
pretrain(G, D)

Epoch 1/1
38000/38000 [==============================] - 63s - loss: 0.0087    


In [8]:
# Get sample noise for the GAN training cycle
def sample_noise(G, noise_dim=16, n_samples=total_samps):
    #X = np.random.uniform(0, 108, size=[n_samples, noise_dim])
    X = get_random_in_here(n_samples,noise_dim)
    y = np.zeros((n_samples, 2))
    y[:, 1] = 1
    return X, y

# Training the GAN by backpropagating the losses of the discriminator on the generated samples, 
# whilst freezing discriminator weights

def train(GAN, G, D, epochs=50, n_samples=total_samps, noise_dim=16, batch_size=100, verbose=False, v_freq=1):
    d_loss = []
    g_loss = []
    e_range = range(epochs)
    if verbose:
        e_range = tqdm(e_range)
    #For each EPOCH
    for epoch in e_range:
        # Get True and Generated Samples with Labels
        X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
        # Allow Discriminator to be Trained
        set_trainability(D, True)
        # Evaluate the Discriminator loss on this set(only to record loss)
        d_loss.append(D.train_on_batch(X, y))
        
        # Now just get the generated samples
        X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
        # Freeze the Discriminator Weights
        set_trainability(D, False)
        # Evaluate the Generator loss on this set
        g_loss.append(GAN.train_on_batch(X, y))
        if verbose and (epoch + 1) % v_freq == 0:
            print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(epoch + 1, g_loss[-1], d_loss[-1]))
    return d_loss, g_loss


In [ ]:
d_loss, g_loss = train(GAN, G, D, verbose=True)

In [ ]:
def piano_roll_to_pretty_midi(piano_roll, sf=16, program_num=1):
    """Convert piano roll to a single instrument pretty_midi object"""
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program_num)
 
    #prepend,append zeros so we can acknowledge inital and ending events
    piano_roll = np.hstack((np.zeros((notes, 1)),
                                 piano_roll,
                                 np.zeros((notes, 1))))
 
    velocity_changes = np.nonzero(np.diff(piano_roll).T)
    current_velocities = np.zeros(notes,dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        velocity = piano_roll[note, time + 1]
        time = time / sf
        if velocity > 0:
            if current_velocities[note] == 0:
                #print('note {} on'.format(pretty_midi.note_number_to_name(note)))
                #print('starting at time {} with velocity {}'.format(time,velocity))
                note_on_time[note] = time
                current_velocities[note] = velocity
            elif current_velocities[note] > 0:
                #change velocity with a special MIDI message
                pass
        else:
            #print('note {} off'.format(pretty_midi.note_number_to_name(note)))
            #print('ending at time {}'.format(time))
            pm_note = pretty_midi.Note(
            velocity=current_velocities[note],
            pitch=note,
            start=note_on_time[note],
            end=time)
            instrument.notes.append(pm_note)
            current_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [ ]:
#Shreyans is a fucking asshole. Dipshit lil bitch. 
def fuck_shrey(d):
    d = np.round(d).astype(int)
    j = np.zeros((128,np.shape(d)[1]))
    i=0
    while (i<np.shape(d)[1]):
        j[d[0,i],i] = 1
        i = i+1
    return j

def pianoroll_to_midi(piano_roll, fs=16, program=0):
    '''Converts a Piano Roll array to a PrettyMidi object
     with a single instrument.
    
    Input:
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    
    Returns:
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    period=1./fs
    
    notes, frames = piano_roll.shape #get number of frames in our piano roll
    pm = pretty_midi.PrettyMIDI() #create a Pretty Midi object
    instrument = pretty_midi.Instrument(program=0) #specify our instrument

    #record previous pitch/velocity so we can concatenate notes together
    prev_pitch=0
    prev_velocity=0
    notelength=0
    starttime=0
    endtime=period
    
    i=0
    while i<frames: #range over the frames of the piano roll

        #need to specify velocity (100 for note, 0 for rest),  start time, end time, and pitch
        
        
        #for ith column of piano roll get the (possible) non-zero index which
        #corresponds to the pitch
        col=piano_roll[:,i] 
        colnext=piano_roll[:,i+1]
        tmp=np.nonzero(col)
        tmpnext=np.nonzero(colnext)
        
        if tmp[0].size==0: #current is rest note, don't need to worry about length
            
            velocity=0
            starttime=i*period
            endtime=period+i*period
            current_pitch=0
           
        
        else:
            #get current pitch and set the start time
            velocity=100
            current_pitch=tmp[0][0]
            starttime=i*period
            
            
            #loop over future notes to find when pitch changes
            pitchchange=False
            while pitchchange==False:
                
                #if end of song quit
                if i==frames-1:
                    endtime=period+i*period
                    break
                    
                #get next note
                colnext=piano_roll[:,i+1]
                tmpnext=np.nonzero(colnext)
                
                #if next note is a rest
                if tmpnext[0].size==0: 
                    endtime=period+i*period
                    break
                    
                #if next frame has different pitch
                elif tmpnext[0][0]!=current_pitch: 
                    endtime=period+i*period
                    pitchchange=True
            
                else:
                    #increment to next frame
                    i=i+1
                    
            
        pm_note=pretty_midi.Note(velocity=100, pitch=current_pitch, start=starttime, end=endtime)
        i=i+1
        #we have appended note, now move to next note
        instrument.notes.append(pm_note)
        #print starttime
        #print endtime
        
    pm.instruments.append(instrument)

    return pm

In [ ]:
def to_the_good_range(string_of_generated_floats):
    OldMin = min(string_of_generated_floats)
    OldMax = max(string_of_generated_floats)

    NewMin = data_minim
    NewMax = data_maxim
    
#     print(" Old Min is ",OldMin)
#     print(" Old Max is ",OldMax)
#     print(" New Min is ",NewMin)
#     print(" New Max is ",NewMax)
    
    OldRange = (OldMax - OldMin)  
    NewRange = (NewMax - NewMin)  

    fixed = []
    for single in string_of_generated_floats:
        new_single = int((((single - OldMin) * NewRange) / OldRange) + NewMin)
        fixed.append(new_single)
    return fixed

#Generating the tune and writing to file
def generate_tune(gan_output):
#     form = UN_normalize_data(gan_output)
    gan_output_UN = to_the_good_range(gan_output)
    form = np.zeros((128,len(gan_output_UN)))
    for i in range(len(gan_output_UN)):
        form[int(gan_output_UN[i])][i]=60
    form_midi = piano_roll_to_pretty_midi(form,8,0)
    form_midi.write('yoyo.mid')
    return form_midi

# Make Music Function
def make_midi(dble_octvs):
    xyz = []
    for i in range(dble_octvs):
        xyz.append(get_random_in_here(1,16))
    xyz_concat = []
    for sixteen_notes in xyz:
        xyz_concat.extend(G.predict(sixteen_notes)[0])
    generate_tune(xyz_concat)

In [ ]:
#how many double octaves(16 notes) are to be generated?
double_octaves = 10
make_midi(double_octaves)

# DONT RUN THIS BITCH

In [ ]:
a = get_random_in_here(1,16)

In [ ]:
b = get_random_in_here(1,16)

In [ ]:
a

In [ ]:
b

In [ ]:
a_gen = G.predict(a)
b_gen = G.predict(b)

In [ ]:
form.shape

In [ ]:
form_midi = piano_roll_to_pretty_midi(form,16,0)
form_midi.write('New_AI_2.mid')

In [ ]:
s = make_midi(10)

In [ ]:
r = make_midi(10)

In [ ]:
xyz=[]
for i in range(5):
    xyz.append(get_random_in_here(1,16))
xyz_concat = []
for sixteen_notes in xyz:
    xyz_concat.extend(G.predict(sixteen_notes)[0])


In [ ]:
# Make Music Function
def make_midi(dble_octvs):
    xyz = []
    for i in range(dble_octvs):
        xyz.append(get_random_in_here(1,16))
    xyz_concat = []
    for sixteen_notes in xyz:
        xyz_concat.extend(G.predict(sixteen_notes)[0])
        return xyz_concat

In [ ]:
D = make_midi(10)

In [ ]:
E = make_midi(10)

In [ ]:
# def to_the_good_range(string_of_generated_floats):
#     OldMin = min(string_of_generated_floats)
#     OldMax = max(string_of_generated_floats)

#     NewMin = data_minim
#     NewMax = data_maxim
    
#     print(" Old Min is ",OldMin)
#     print(" Old Max is ",OldMax)
#     print(" New Min is ",NewMin)
#     print(" New Max is ",NewMax)
    
#     OldRange = (OldMax - OldMin)  
#     NewRange = (NewMax - NewMin)  

#     fixed = []
#     for single in string_of_generated_floats:
#         new_single = int((((single - OldMin) * NewRange) / OldRange) + NewMin)
#         fixed.append(new_single)
#     return fixed
    
#     form = np.zeros((128,len(fixed)))
#     for i in range(len(fixed)):
#         form[int(fixed[i])][i]=60
#     return form

# #Generating the tune and writing to file
# def generate_tune(gan_output):
#     form = to_the_good_range(gan_output)
#     form_midi = piano_roll_to_pretty_midi(form,16,0)
#     form_midi.write('TIMEPASS.mid')
#     return form_midi

In [ ]:
D_f = to_the_good_range(D)
E_f = to_the_good_range(E)

In [ ]:
D_f==E_f

In [ ]:
E_f

In [ ]:
D_f

In [ ]:
XN_noise

In [ ]:
XN_noise_2

In [ ]:
XN_noise = get_random_in_here(1,16)

In [ ]:
XN_noise

In [ ]:
A,B =sample_data_and_gen(G)

In [ ]:
def get_ALL_data(n_samples):
    return ALL_DATA[:n_samples]

In [ ]:
ALL_NEW_DATA[34]

In [ ]:
data_minim

In [ ]:
ALL_DATAx = normalize_data(ALL_DATA)

In [ ]:
ALL_DATAy = UN_normalize_data(ALL_DATAx)

In [ ]:
ALL_DATAy == ALL_DATA

In [ ]:
def sample_data(training_path):
    filenames = []
    lens = []
    fin_midi_stream = []
    for filename in os.listdir(training_path):
        if filename.endswith('.mid'):
            filenames.append(os.path.join(training_path,filename))
    fin_midi_streama = []
    fin_midi_files = []
    for file in filenames:
        temp = pretty_midi.PrettyMIDI(file)
        melody = temp.instruments[0]
    #     chord = temp.instruments[1]
        melody_math = melody.get_piano_roll(16)
        curr_stream = np.argmax(melody_math, axis=0)
        fin_midi_streama.extend(curr_stream)
    chop_size = 256
    num_of_chops = len(fin_midi_streama)//chop_size
    for i in range(num_of_chops):
        fin_midi_files.append(fin_midi_streama[(i)*chop_size:(i+1)*chop_size])
    
    return fin_midi_files

In [ ]:
d = sample_data(training_path)

In [ ]:
ALL_NEW_DATA[0]

In [ ]:
data_minim = min(ALL_DATA[0])
data_maxim = max(ALL_DATA[0])

In [ ]:
data_maxim

In [ ]:
e = normalize_data(ALL_DATA)

In [ ]:
f = UN_normalize_data(e)

In [ ]:
data_minim = min(min(ALL_DATA))
data_maxim = max(max(ALL_DATA))

In [ ]:
s,d = sample_data_and_gen(G)

In [ ]:
len(s)

In [ ]:
def make_midi(dble_octvs):
    xyz = []
    for i in range(dble_octvs):
        xyz.append(get_random_in_here(1,16))
    xyz_concat = []
    for sixteen_notes in xyz:
        xyz_concat.extend(G.predict(sixteen_notes)[0])
    return xyz_concat

In [ ]:
def to_the_good_range(string_of_generated_floats):
    OldMin = min(string_of_generated_floats)
    OldMax = max(string_of_generated_floats)

    NewMin = data_minim
    NewMax = data_maxim
    
    print(" Old Min is ",OldMin)
    print(" Old Max is ",OldMax)
    print(" New Min is ",NewMin)
    print(" New Max is ",NewMax)
    
    OldRange = (OldMax - OldMin)  
    NewRange = (NewMax - NewMin)  

    fixed = []
    for single in string_of_generated_floats:
        new_single = int((((single - OldMin) * NewRange) / OldRange) + NewMin)
        fixed.append(new_single)
    return fixed

In [ ]:
r[:10]

In [ ]:
q,w = sample_data_and_gen(G)

In [ ]:
q

In [ ]:
ALL_NEW_DATA = normalize_data(ALL_DATA)

In [ ]:
len(ALL_DATA)